# Imports

In [1]:
from gensim.models import KeyedVectors

In [ ]:
pt_model = KeyedVectors.load_word2vec_format('wiki.pt/wiki.pt.vec')

In [ ]:
words = []
for word in en_model.vocab:
    words.append(word)

In [ ]:
print("Number of Tokens: {}".format(len(words)))

In [ ]:
print("Dimension of a word vector: {}".format(
    len(en_model[words[0]])
))

In [ ]:
print("Vector components of a word: {}".format(
    en_model[words[0]]
))

In [ ]:
find_similar_to = 'car'

for similar_word in en_model.similar_by_word(find_similar_to):
    print("Word: {0}, Similarity: {1:.2f}".format(
        similar_word[0], similar_word[1]
    ))

In [ ]:
word_add = ['dhaka', 'india']
word_sub = ['bangladesh']


for resultant_word in en_model.most_similar(
    positive=word_add, negative=word_sub
):
    print("Word : {0} , Similarity: {1:.2f}".format(
        resultant_word[0], resultant_word[1]
    ))